Due Date: December 7th

# Vaccine Development with Dynamic Programming

You are the CEO of a biotech company which is considering the development of a new vaccine. Starting at phase 0 (state 0), the drug develpment can stay in the same state or advance to "phase 1  with promising results" (state 1) or advance to "phase 1 with disappointing results" (state 2), or fail completely (state 4). At phase 1, the drug can stay in the same state, fail or become a success (state 3), in which case you will sell its patent to a big pharma company for $\$10$ million.
These state transitions happen from month to month, and at each state, you have the option to make an additional investment of \$100,000, which increases the chances of success.

After careful study, your analysts develop the program below to simulate different scenarios using statistical data from similar projects. 

Use a discount factor of 0.996.

- 1) Write a policy iteration algorithm to compute the value of this project. Please print the full V vector.

- 2 )Write a value iteration algorithm to compute the value of this project. Please print the full V vector.

In [6]:
import numpy as np


class MDP():
    def __init__(self):
        self.A = [0, 1]
        self.S = [0, 1, 2, 3, 4]

        P0 = np.array([[0.5, .15, .15, 0, .20],
                       [0, .5, .0, .25, .25],
                       [0, 0, .15, .05, .8],
                       [0, 0, 0, 0, 1],
                       [0, 0, 0, 0, 1]])

        R0 = np.array([0, 0, 0, 10, 0])

        P1 = np.array([[0.5, .25, .15, 0, .10],
                       [0, .5, .0, .35, .15],
                       [0, 0, .20, .05, .75],
                       [0, 0, 0, 0, 1],
                       [0, 0, 0, 0, 1]])

        R1 = np.array([-0.1, -0.1, -0.1, 10, 0])

        self.P = [P0, P1]
        self.R = [R0, R1]

    def step(self, s, a):
        s_prime = np.random.choice(len(self.S), p=self.P[a][s])
        R = self.R[a][s]
        if s_prime == 4:
            done = True
        else:
            done = False
        return s_prime, R, done

    def simulate(self, s, a, π):
        done = False
        t = 0
        history = []
        while not done:
            if t > 0:
                a = π[s]
            s_prime, R, done = self.step(s, a)
            history.append((s, a, R))
            s = s_prime
            t += 1

        return history

You can access the transition probability matrices and the reward vector as follows:

In [7]:
mdp = MDP()
P = mdp.P
R = mdp.R


s = 2  # current state
s_prime = 4  # next state
a = 1  # chosen action

# Probability of transition from state s (2) to s_prime (4) if action == a (1):
print(P[a][s, s_prime])

# Reward at state s if action = a
print(R[a][s])

0.75
-0.1


# Policy Iteration

In [8]:
π = [0, 0, 0,0,0]
γ = 0.996

def construct_Rπ(R, π, S):
    Rπ = np.zeros(len(S))
    for s in S:
        Rπ[s] = R[π[s]][s]
    return Rπ


def construct_Pπ(P, π, S):
    Pπ = np.zeros((len(S), len(S)))
    for s in S:
        for s_prime in S:
            Pπ[s, s_prime] = P[π[s]][s, s_prime]
    return Pπ

def policy_evaluation(π, Vπ):
    Rπ = construct_Rπ(mdp.R, π, mdp.S)
    Pπ = construct_Pπ(mdp.P, π, mdp.S)
    for iteration in range(1): #把這邊次數減少 
        Vπ = Rπ + γ * Pπ @ Vπ
    return Vπ


def policy_improvement(Vπ):
    # Compute Qπ using Vπ
    Qπ = np.zeros((5, 2))
    π_prime = np.zeros(5, dtype=np.int32)
    for s in mdp.S:
        for a in mdp.A:
            Qπ[s, a] = R[a][s] + γ * P[a][s] @ Vπ

    # Greedy updates
    for s in mdp.S:
        π_prime[s] = np.argmax(Qπ[s, :])
    return π_prime

In [9]:
Vπ = np.zeros(5)
for iteration in range(100): #然後這邊次數增大
    Vπ = policy_evaluation(π, Vπ)
    π = policy_improvement(Vπ)
print(Vπ)

[ 3.32067538  6.74501992  0.58546908 10.          0.        ]


# Value Iteration

In [10]:
v0 = np.zeros(5)
γ = 0.996

def update(v0):
    v1 = np.zeros_like(v0)
    for s in mdp.S:
        v1[s] = np.max([R[a][s] + γ * P[a][s] @ v0 for a in mdp.A])
    return v1


for _ in range(2000):
    v1 = update(v0)
    error = np.abs(v0 - v1).max()
    if error < 1e-10:
        break
    v0 = v1
print(v1)

[ 3.32067538  6.74501992  0.58546908 10.          0.        ]
